In [1]:
import requests
import json
from pathlib import Path
import os
import sys
import pandas as pd
from datetime import datetime
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

import utils as util

In [2]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo" 
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [4]:
requestData = {
 
    # SiteName takes in two different parameters: [CameraOne, CameraTwo]
    # SceneName takes in different parameteres depending on SiteName:  [CameraOne [HospitalView]], [CameraTwo [SmallStreetSlide, StreetSlide, SchoolView]]

    # If you want to return all availalbe scans and images, comment out the Coordinates line.
   
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2022-01-01T00:00:00.000Z',
        'Maximum': '2023-02-27T15:43:16.000Z'
    },
    #'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  # comment out if you want to return all images
}

In [5]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [6]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [7]:
channelInfoData = channelResponse.json()
data = response.json()

In [8]:
# obtain measurement labels
skip_rate = 2
measurement_labels = []
for item in channelInfoData:
    for index, (key, value) in enumerate(item.items()):
        if index % skip_rate == 0:
            measurement_labels.append(value)

# map labels with measurements and drop redundant columns
df = pd.DataFrame(data)
count = 0
for i in measurement_labels:
    df[i] =df['Values'].apply(lambda x: x[count] )
    count = count + 1
df = df.drop(columns=['Values'])

# Put it into function 
We wrap the data request into a nice reusable function and store it under the src folder and cann simply call it in the next step.

In [9]:
camera_one_data = util.return_data()
camera_one_data.columns

Index(['Timestamp', 'ImageUrls', 'ACTUAL_TEMPERATURE', 'HUMIDITY',
       'ILLUMINATION', 'RAIN_COUNTER', 'SUNSHINEDURATION', 'WIND_SPEED',
       'WIND_SPEED_CALC', 'Rain.Day.Counter', 'Rain.Diff',
       'Sunshine.Day.Counter', 'Sunshine.Diff', 'Site_Scence'],
      dtype='object')

In [10]:
camera_one_data

,Timestamp,ImageUrls,ACTUAL_TEMPERATURE,HUMIDITY,ILLUMINATION,RAIN_COUNTER,SUNSHINEDURATION,WIND_SPEED,WIND_SPEED_CALC,Rain.Day.Counter,Rain.Diff,Sunshine.Day.Counter,Sunshine.Diff,Site_Scence
0,2023-01-18T15:47:13.68219+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,17.100000,67.0,2799.000000,736.000000,218.0,13.200000,3.666667,0.0,0.0,542,0.0,CameraOne_HospitalView
1,2023-01-18T16:00:00.026914+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,17.100000,67.0,1324.099976,736.000000,218.0,10.600000,2.944444,0.0,0.0,542,0.0,CameraOne_HospitalView
2,2023-01-19T09:00:00.464237+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,19.799999,52.0,100000.000000,751.400024,487.0,39.700001,11.027778,0.0,0.0,182,3.0,CameraOne_HospitalView
3,2023-01-19T16:00:00.012271+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,17.500000,71.0,122.000000,751.400024,850.0,9.500000,2.638889,0.0,0.0,545,0.0,CameraOne_HospitalView
4,2023-01-20T09:00:00.005397+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,18.400000,70.0,5659.000000,753.099976,1122.0,16.299999,4.527778,0.0,0.0,180,3.0,CameraOne_HospitalView
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2023-02-25T14:00:00.427186+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,16.299999,77.0,2054.000000,814.200012,5558.0,7.000000,1.944444,5.3,0.0,474,0.0,CameraOne_HospitalView
78,2023-02-26T06:00:00.011235+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,16.299999,77.0,2054.000000,814.200012,5558.0,7.000000,1.944444,5.3,0.0,474,0.0,CameraOne_HospitalView
79,2023-02-26T14:00:00.449884+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,16.299999,77.0,2054.000000,814.200012,5558.0,7.000000,1.944444,5.3,0.0,474,0.0,CameraOne_HospitalView
80,2023-02-27T06:00:00.018633+00:00,[https://docucamrw.hesotech.eu/DocuCam/CameraO...,16.299999,77.0,2054.000000,814.200012,5558.0,7.000000,1.944444,5.3,0.0,474,0.0,CameraOne_HospitalView


In [ ]:
# get all datasets from camera two
camera_two_data_1 = util.return_data(site = "CameraTwo", scene = "SmallFieldSlide", all_coordinates = True)
camera_two_data_2 = util.return_data(site = "CameraTwo", scene = "StreetSlide", all_coordinates = True)
camera_two_data_3 = util.return_data(site = "CameraTwo", scene = "SchoolView", all_coordinates = True)

In [ ]:
# merge all four datasets and sort by 

# To-Do
* allow for coordinate input in query function
* allow to query all different datasets for each site and each scence and combine